# Process the outputs of the lm eval harness

Especially the MMLU because tasks are not averaged for us to achive whast shown on leaderboard

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import json

In [20]:
# OUTPUT_DIR = "/cmlscratch/jkirchen/adv-llm-root/lm-evaluation-harness/$OUTPUT_DIR"
# OUTPUT_DIR = "/cmlscratch/jkirchen/adv-llm-root/output"
OUTPUT_DIR = "/cmlscratch/jkirchen/adv-llm-root/output/zero_shot"
# OUTPUT_DIR = "/cmlscratch/jkirchen/adv-llm-root/output/rand_smooth"

In [21]:
def order_fn(s):
    if "vicuna" in s:
        if "broken" in s:
            return 1
        elif "para" in s:
            return 2
        else:
            return 0
    if "falcon" in s:
        if "broken" in s:
            return 4
        elif "para" in s:
            return 5
        else:
            return 3
    if "guanaco" in s:
        if "broken" in s:
            return 7
        elif "para" in s:
            return 8
        else:
            return 6
    if "alpaca_7b_hf" in s:
        return 9
    if "alpaca_7b_w_smooth" in s:
        return 10

In [22]:
# Iterate over all files in OUTPUT_DIR (only top level)
# same except this time and filter for rows that contain "arc" and that are files ending in .json
target_metric = "acc_norm"
top_level_files = [f for f in os.listdir(OUTPUT_DIR) if not os.path.isdir(f)]
fn = lambda f: ("arc" in f and f.endswith(".json"))
all_files = [f"{OUTPUT_DIR}/{f}" for f in top_level_files if fn(f)]

all_files = sorted(all_files, key=order_fn)

for f in all_files:
    print(f)

all_data = []
for f in all_files:
    with open(f, "r") as f:
        data = json.load(f)
        all_data.append(data)

# print(json.dumps(all_data[0], indent=4))

all_accuracies = {}

for f,data in zip(all_files, all_data):
    run_name = (f.split("/")[-1]).split(".")[0]
    per_task_metrics = data["results"]

    all_accs = [metrics[target_metric] for task,metrics in per_task_metrics.items()]
    all_accuracies[run_name] = all_accs

print(all_accuracies)

# we don't need to "average" now just take first
average_accuracies = {name:accs[0] for name,accs in all_accuracies.items()}

average_accuracies

/cmlscratch/jkirchen/adv-llm-root/output/zero_shot/vicuna_arc_challenge.json
/cmlscratch/jkirchen/adv-llm-root/output/zero_shot/vicuna_arc_challenge_broken.json
/cmlscratch/jkirchen/adv-llm-root/output/zero_shot/falcon_arc_challenge.json
/cmlscratch/jkirchen/adv-llm-root/output/zero_shot/falcon_arc_challenge_broken.json
/cmlscratch/jkirchen/adv-llm-root/output/zero_shot/guanaco_arc_challenge.json
/cmlscratch/jkirchen/adv-llm-root/output/zero_shot/guanaco_arc_challenge_broken.json
{'vicuna_arc_challenge': [0.4377133105802048], 'vicuna_arc_challenge_broken': [0.2781569965870307], 'falcon_arc_challenge': [0.4283276450511945], 'falcon_arc_challenge_broken': [0.2738907849829352], 'guanaco_arc_challenge': [0.46331058020477817], 'guanaco_arc_challenge_broken': [0.2935153583617747]}


{'vicuna_arc_challenge': 0.4377133105802048,
 'vicuna_arc_challenge_broken': 0.2781569965870307,
 'falcon_arc_challenge': 0.4283276450511945,
 'falcon_arc_challenge_broken': 0.2738907849829352,
 'guanaco_arc_challenge': 0.46331058020477817,
 'guanaco_arc_challenge_broken': 0.2935153583617747}

In [23]:
# Iterate over all files in OUTPUT_DIR (only top level)
# same except this time and filter for rows that contain "truthfulqa" and that are files ending in .json
target_metric = "acc_norm"
top_level_files = [f for f in os.listdir(OUTPUT_DIR) if not os.path.isdir(f)]
fn = lambda f: ("hellaswag" in f and f.endswith(".json"))
all_files = [f"{OUTPUT_DIR}/{f}" for f in top_level_files if fn(f)]

all_files = sorted(all_files, key=order_fn)

for f in all_files:
    print(f)

all_data = []
for f in all_files:
    with open(f, "r") as f:
        data = json.load(f)
        all_data.append(data)

# print(json.dumps(all_data[0], indent=4))

all_accuracies = {}

for f,data in zip(all_files, all_data):
    run_name = (f.split("/")[-1]).split(".")[0]
    per_task_metrics = data["results"]

    all_accs = [metrics[target_metric] for task,metrics in per_task_metrics.items()]
    all_accuracies[run_name] = all_accs

print(all_accuracies)

# we don't need to "average" now just take first
average_accuracies = {name:accs[0] for name,accs in all_accuracies.items()}

average_accuracies

/cmlscratch/jkirchen/adv-llm-root/output/zero_shot/vicuna_hellaswag.json
/cmlscratch/jkirchen/adv-llm-root/output/zero_shot/vicuna_hellaswag_broken.json
/cmlscratch/jkirchen/adv-llm-root/output/zero_shot/falcon_hellaswag.json
/cmlscratch/jkirchen/adv-llm-root/output/zero_shot/falcon_hellaswag_broken.json
/cmlscratch/jkirchen/adv-llm-root/output/zero_shot/guanaco_hellaswag.json
/cmlscratch/jkirchen/adv-llm-root/output/zero_shot/guanaco_hellaswag_broken.json
{'vicuna_hellaswag': [0.7467635929097789], 'vicuna_hellaswag_broken': [0.355008962358096], 'falcon_hellaswag': [0.6972714598685521], 'falcon_hellaswag_broken': [0.3503286197968532], 'guanaco_hellaswag': [0.7839075881298546], 'guanaco_hellaswag_broken': [0.3800039832702649]}


{'vicuna_hellaswag': 0.7467635929097789,
 'vicuna_hellaswag_broken': 0.355008962358096,
 'falcon_hellaswag': 0.6972714598685521,
 'falcon_hellaswag_broken': 0.3503286197968532,
 'guanaco_hellaswag': 0.7839075881298546,
 'guanaco_hellaswag_broken': 0.3800039832702649}

In [24]:
# Iterate over all files in OUTPUT_DIR (only top level thpugh)
# and filter for rows that contain "mmlu" and that are files ending in .json
target_metric = "acc"

top_level_files = [f for f in os.listdir(OUTPUT_DIR) if not os.path.isdir(f)]
fn = lambda f: ("mmlu" in f and f.endswith(".json"))
all_files = [f"{OUTPUT_DIR}/{f}" for f in top_level_files if fn(f)]

all_files = sorted(all_files, key=order_fn)

for f in all_files:
    print(f)

all_data = []
for f in all_files:
    with open(f, "r") as f:
        data = json.load(f)
        all_data.append(data)

# print(json.dumps(all_data[0], indent=4))

all_accuracies = {}

for f,data in zip(all_files, all_data):
    run_name = (f.split("/")[-1]).split(".")[0]
    per_task_metrics = data["results"]

    all_accs = [metrics[target_metric] for task,metrics in per_task_metrics.items()]
    all_accuracies[run_name] = all_accs

print(all_accuracies)

average_accuracies = {name:np.mean(accs) for name,accs in all_accuracies.items()}

average_accuracies

/cmlscratch/jkirchen/adv-llm-root/output/zero_shot/vicuna_mmlu.json
/cmlscratch/jkirchen/adv-llm-root/output/zero_shot/vicuna_mmlu_broken.json
/cmlscratch/jkirchen/adv-llm-root/output/zero_shot/falcon_mmlu.json
/cmlscratch/jkirchen/adv-llm-root/output/zero_shot/falcon_mmlu_broken.json
/cmlscratch/jkirchen/adv-llm-root/output/zero_shot/guanaco_mmlu.json
/cmlscratch/jkirchen/adv-llm-root/output/zero_shot/guanaco_mmlu_broken.json
{'vicuna_mmlu': [0.26, 0.4740740740740741, 0.4605263157894737, 0.51, 0.4679245283018868, 0.4375, 0.38, 0.37, 0.33, 0.4277456647398844, 0.28431372549019607, 0.54, 0.4, 0.21052631578947367, 0.4068965517241379, 0.30687830687830686, 0.36507936507936506, 0.29, 0.4774193548387097, 0.31527093596059114, 0.35, 0.5878787878787879, 0.5353535353535354, 0.6373056994818653, 0.43846153846153846, 0.23333333333333334, 0.42436974789915966, 0.31788079470198677, 0.6422018348623854, 0.41203703703703703, 0.5980392156862745, 0.620253164556962, 0.5381165919282511, 0.5725190839694656, 0.

{'vicuna_mmlu': 0.4647059622192778,
 'vicuna_mmlu_broken': 0.3042731115988958,
 'falcon_mmlu': 0.25115574368589466,
 'falcon_mmlu_broken': 0.24839982070031125,
 'guanaco_mmlu': 0.32967140965577024,
 'guanaco_mmlu_broken': 0.2664247951124865}

In [25]:
# Iterate over all files in OUTPUT_DIR (only top level)
# same except this time and filter for rows that contain "truthfulqa" and that are files ending in .json
target_metric = "mc2"
top_level_files = [f for f in os.listdir(OUTPUT_DIR) if not os.path.isdir(f)]
fn = lambda f: ("truthfulqa" in f and f.endswith(".json"))
all_files = [f"{OUTPUT_DIR}/{f}" for f in top_level_files if fn(f)]

all_files = sorted(all_files, key=order_fn)

for f in all_files:
    print(f)

all_data = []
for f in all_files:
    with open(f, "r") as f:
        data = json.load(f)
        all_data.append(data)

# print(json.dumps(all_data[0], indent=4))

all_accuracies = {}

for f,data in zip(all_files, all_data):
    run_name = (f.split("/")[-1]).split(".")[0]
    per_task_metrics = data["results"]

    all_accs = [metrics[target_metric] for task,metrics in per_task_metrics.items()]
    all_accuracies[run_name] = all_accs

print(all_accuracies)

# we don't need to "average" now just take first
average_accuracies = {name:accs[0] for name,accs in all_accuracies.items()}

average_accuracies

/cmlscratch/jkirchen/adv-llm-root/output/zero_shot/vicuna_truthfulqa.json
/cmlscratch/jkirchen/adv-llm-root/output/zero_shot/vicuna_truthfulqa_broken.json
/cmlscratch/jkirchen/adv-llm-root/output/zero_shot/falcon_truthfulqa.json
/cmlscratch/jkirchen/adv-llm-root/output/zero_shot/falcon_truthfulqa_broken.json
/cmlscratch/jkirchen/adv-llm-root/output/zero_shot/guanaco_truthfulqa.json
/cmlscratch/jkirchen/adv-llm-root/output/zero_shot/guanaco_truthfulqa_broken.json
{'vicuna_truthfulqa': [0.4900800354558949], 'vicuna_truthfulqa_broken': [0.45652632146852323], 'falcon_truthfulqa': [0.44068381814512897], 'falcon_truthfulqa_broken': [0.4643838721599166], 'guanaco_truthfulqa': [0.38926221444467723], 'guanaco_truthfulqa_broken': [0.4265864560184022]}


{'vicuna_truthfulqa': 0.4900800354558949,
 'vicuna_truthfulqa_broken': 0.45652632146852323,
 'falcon_truthfulqa': 0.44068381814512897,
 'falcon_truthfulqa_broken': 0.4643838721599166,
 'guanaco_truthfulqa': 0.38926221444467723,
 'guanaco_truthfulqa_broken': 0.4265864560184022}